In [1]:
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib
import traceback

from tqdm import tqdm
import pandas as pd



from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

In [75]:
!{sys.executable} -m pip install -r requirements.txt

  Using cached pandas-1.2.4-cp37-cp37m-manylinux1_x86_64.whl (9.9 MB)
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
     |████████████████████████████████| 15.3 MB 763 kB/s eta 0:00:01


In [26]:
class Args():
    def __init__(self, term='restaurants', location='Philadelphia, Pa', streets_path='street_name.txt', data_path= 'data.pkl',iteration=0):
        
        self.term=term
        self.location=location
        self.street_path=street_path
        self.data_path=data_path
        self.iteration=iteration
        
    def init_parsearges(self):
        ap = argparse.ArgumentParser()
        
        # Argument of insightface
        ap.add_argument("--term", default=self.term,
                        type=str, help='Search term')
        
        ap.add_argument("--location", default=self.location,
                        type=str, help='Search location')
        ap.add_argument("--street_path", default=self.street_path,
                        type=str, help='path to names of the streets')
        ap.add_argument("--data_path", default=self.data_path,
                        type=str, help='path to save data')
        
        ap.add_argument("--iteration", default=self.iteration,
                         type=int, help='starting iteration')

        args = ap.parse_args()
        
        return args


# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
API_KEY= 'cwhlm9INXDobbgkfHlDhVVVVrncPHYNw9DoTcVLXuHkeCcGVVj7F7iPgMqaeW2JH5fKS24dRBQJoIkKJfLLgpJCGbPLmMuHSIqHsFsyyggXyzcohMWKSajdxGOHfX3Yx' 


# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'restaurants'
DEFAULT_LOCATION = 'Philadelphia, Pa'
SEARCH_LIMIT = 50


def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        dict: The JSON response from the request.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

#     print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location):
    """Query the Search API by a search term and location.

    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.

    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(api_key, business_id):
    """Query the Business API by a business ID.

    Args:
        business_id (str): The ID of the business to query.

    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)


def query_api(term, location):
    """Queries the API by the input values from the user.

    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)
    return response

    businesses = response.get('businesses')

    if not businesses:
        print(response)
        return []

#     business_id = businesses[0]['id']

#     print(u'{0} businesses found, querying business info ' \
#         'for the top result "{1}" ...'.format(
#             len(businesses), business_id))
#     response = get_business(API_KEY, business_id)

#     data = [get_business(API_KEY, business['id']) for business in businesses ]
#     print(u'Result for business "{0}" found:'.format(business_id))
#     pprint.pprint(response, indent=2)
    return businesses

def read_street(path='street_name.txt'):
    with open(path, 'r') as f:
        streets = f.readlines()
    return [street.replace("\n","") for street in streets]

def read_data(path='data.pkl'):
    return pd.read_pickle(path)

def main(term='restaurants', location='Philadelphia, Pa', streets_path='street_name.txt', data_path= 'data.pkl',iteration=0):
    
#     parser = argparse.ArgumentParser()

#     parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM,
#                         type=str, help='Search term (default: %(default)s)')
#     parser.add_argument('-l', '--location', dest='location',
#                         default=DEFAULT_LOCATION, type=str,
#                         help='Search location (default: %(default)s)')

#     input_values = parser.parse_args()


    streets = read_street(path=streets_path)
    data=[] if iteration==0 else read_data(data_path).to_dict(orient='records')
    for (i, street) in tqdm(enumerate(streets)):
        if i>=iteration: 
            address = street +' '+ location
            try:
                data.extend(query_api(term, address))
            except:
                df = pd.DataFrame(data)
                df.to_pickle(data_path)
                traceback.print_exc()
                sys.exit()
        else:
            pass
        
    print("-------------------SAVING DATA---------------------")
    df = pd.DataFrame(data)
    df.to_pickle(data_path)
    
    return df

In [4]:
term='restaurants'
location='Philadelphia, Pa'
r=query_api(term, location)

In [24]:
r.get('businesses')[0]

{'id': '6ajnOk0GcY9xbb5Ocaw8Gw',
 'alias': 'barbuzzo-philadelphia',
 'name': 'Barbuzzo',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/88mJSIDPi3JUsCK2FsVJqg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/barbuzzo-philadelphia?adjust_creative=LYlilwjYqjn0f0LIANnrQg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LYlilwjYqjn0f0LIANnrQg',
 'review_count': 2772,
 'categories': [{'alias': 'mediterranean', 'title': 'Mediterranean'},
  {'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'italian', 'title': 'Italian'}],
 'rating': 4.5,
 'coordinates': {'latitude': 39.94999, 'longitude': -75.16216},
 'transactions': ['pickup', 'delivery'],
 'price': '$$',
 'location': {'address1': '110 S 13th St',
  'address2': '',
  'address3': '',
  'city': 'Philadelphia',
  'zip_code': '19107',
  'country': 'US',
  'state': 'PA',
  'display_address': ['110 S 13th St', 'Philadelphia, PA 19107']},
 'phone': '+12155469300',
 'display_phone': '(215) 546-9300',
 'dist

In [5]:

args=Args(term='restaurants', location='Philadelphia, Pa', streets_path='street_name.txt', data_path= 'data.pkl',iteration=0)
# args=args.init_parsearges()
b=main(term=args.term, location=args.location, streets_path=args.street_path, data_path= args.data_path,iteration=args.iteration)

0it [00:12, ?it/s]


NameError: name 'error' is not defined

# Test

In [136]:
import time
a=[]
a.extend([])
start=time.time()
for i in range(10000):
    a.append(list(range(500)))
end=time.time()
print(end -start)

0.1197962760925293


In [135]:
import time
a=[]
start=time.time()
for i in range(10000):
    a.extend(list(range(500)))
end=time.time()
print(end -start)

0.08768534660339355


In [25]:
a= { 'alias': 'barbuzzo-philadelphia',
  'categories': [ {'alias': 'mediterranean', 'title': 'Mediterranean'},
                  {'alias': 'pizza', 'title': 'Pizza'},
                  {'alias': 'italian', 'title': 'Italian'}],
  'coordinates': {'latitude': 39.94999, 'longitude': -75.16216},
  'display_phone': '(215) 546-9300',
  'hours': [ { 'hours_type': 'REGULAR',
               'is_open_now': False,
               'open': [ { 'day': 0,
                           'end': '2100',
                           'is_overnight': False,
                           'start': '1600'},
                         { 'day': 1,
                           'end': '2100',
                           'is_overnight': False,
                           'start': '1600'},
                         { 'day': 2,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1600'},
                         { 'day': 3,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1600'},
                         { 'day': 4,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1200'},
                         { 'day': 5,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1200'},
                         { 'day': 6,
                           'end': '2100',
                           'is_overnight': False,
                           'start': '1200'}]}],
  'id': '6ajnOk0GcY9xbb5Ocaw8Gw',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/88mJSIDPi3JUsCK2FsVJqg/o.jpg',
  'is_claimed': True,
  'is_closed': False,
  'location': { 'address1': '110 S 13th St',
                'address2': '',
                'address3': '',
                'city': 'Philadelphia',
                'country': 'US',
                'cross_streets': '',
                'display_address': ['110 S 13th St', 'Philadelphia, PA 19107'],
                'state': 'PA',
                'zip_code': '19107'},
  'name': 'Barbuzzo',
  'phone': '+12155469300',
  'photos': [ 'https://s3-media2.fl.yelpcdn.com/bphoto/88mJSIDPi3JUsCK2FsVJqg/o.jpg',
              'https://s3-media1.fl.yelpcdn.com/bphoto/dn5Pfj75I1fWZ2LqZ0nJ4Q/o.jpg',
              'https://s3-media1.fl.yelpcdn.com/bphoto/3hdgyod-3e5eW4rfhvVh0A/o.jpg'],
  'price': '$$',
  'rating': 4.5,
  'review_count': 2772,
  'transactions': ['pickup', 'delivery'],
  'url': 'https://www.yelp.com/biz/barbuzzo-philadelphia?adjust_creative=LYlilwjYqjn0f0LIANnrQg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=LYlilwjYqjn0f0LIANnrQg'}

b= {'id': '6ajnOk0GcY9xbb5Ocaw8Gw',
 'alias': 'barbuzzo-philadelphia',
 'name': 'Barbuzzo',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/88mJSIDPi3JUsCK2FsVJqg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/barbuzzo-philadelphia?adjust_creative=LYlilwjYqjn0f0LIANnrQg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LYlilwjYqjn0f0LIANnrQg',
 'review_count': 2772,
 'categories': [{'alias': 'mediterranean', 'title': 'Mediterranean'},
  {'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'italian', 'title': 'Italian'}],
 'rating': 4.5,
 'coordinates': {'latitude': 39.94999, 'longitude': -75.16216},
 'transactions': ['pickup', 'delivery'],
 'price': '$$',
 'location': {'address1': '110 S 13th St',
  'address2': '',
  'address3': '',
  'city': 'Philadelphia',
  'zip_code': '19107',
  'country': 'US',
  'state': 'PA',
  'display_address': ['110 S 13th St', 'Philadelphia, PA 19107']},
 'phone': '+12155469300',
 'display_phone': '(215) 546-9300',
 'distance': 877.931789849032}